In [20]:
!pip install --upgrade pip
!pip install --upgrade sklearn
!pip install jieba
!pip install pyldavis

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [21]:
import pandas as pd
df = pd.read_csv("datascience.csv", encoding='gb18030')
df.head()

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...


In [22]:
df.shape

(1024, 3)

In [23]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [24]:
df["content_cutted"] = df.content.apply(chinese_word_cut)

In [25]:
df.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [27]:
n_features = 1000

In [28]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [29]:
from sklearn.decomposition import LatentDirichletAllocation

In [40]:
n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=50,
                                learning_method='online',
                                learning_offset=50.0,
                                random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=50,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [41]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


In [42]:
n_top_words = 20

In [43]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
学习 模型 使用 算法 方法 机器 可视化 神经网络 特征 处理 不同 计算 用户 数据库 系统 如果 分类 训练 一种 基于
Topic #1:
这个 就是 可能 没有 如果 他们 自己 很多 什么 不是 但是 或者 因为 时候 这样 现在 电子 一些 所以 孩子
Topic #2:
企业 平台 服务 管理 互联网 数据分析 公司 产品 用户 业务 行业 客户 金融 创新 实现 价值 系统 能力 工作 需求
Topic #3:
中国 2016 市场 增长 10 城市 用户 2015 关注 行业 其中 30 人口 检索 阅读 大众 投资 全国 美国 20
Topic #4:
人工智能 学习 领域 智能 机器人 机器 人类 公司 深度 研究 未来 识别 已经 系统 计算机 目前 医疗 语音 方面 服务



In [44]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.149635  0.021451       1        1  25.852071
2     -0.100942  0.102186       2        1  25.798362
1      0.127037 -0.112524       3        1  21.632325
4     -0.005510  0.118440       4        1  14.625269
3     -0.170219 -0.129553       5        1  12.091972, topic_info=     Term         Freq        Total Category  logprob  loglift
122  人工智能  2771.000000  2771.000000  Default  30.0000  30.0000
364    学习  4194.000000  4194.000000  Default  29.0000  29.0000
148    企业  3220.000000  3220.000000  Default  28.0000  28.0000
615    机器  2178.000000  2178.000000  Default  27.0000  27.0000
586    智能  1450.000000  1450.000000  Default  26.0000  26.0000
..    ...          ...          ...      ...      ...      ...
458    微信   266.676300   493.258458   Topic5  -5.2453   1.4976
845    行业   346.365488  1589.176433   Topic5  -4.9839   0.5892
560    数量   259.832297   467.563261   Topic5  -5.2713   1.5251
97     主要   273.202890  1246.241126   Topic5  -5.2211   0.5950
501    报告   257.056074   617.836739   Topic5  -5.2821   1.2357

[310 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.027158   00
0         5  0.969939   00
1         1  0.186738   10
1         2  0.044899   10
1         3  0.108165   10
...     ...       ...  ...
992       5  0.024959   领域
996       1  0.023137   驾驶
996       3  0.069412   驾驶
996       4  0.896569   驾驶
996       5  0.005784   驾驶

[1027 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4])

In [46]:
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.display(data)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, 
                                max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)


In [47]:
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.display(data)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
